# Preparing libranzas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime, timedelta 
from dateutil import relativedelta
import s3fs
import pyarrow.parquet as pq
import os
import calendar
import gc


pd.set_option('max_columns',None)
pd.set_option('max_rows',None)

%matplotlib inline

In [2]:
BASE_PATH_LIBRANZA = 's3://adl-refined-dev-popular/data_orig/libranzas/'
LIBRANZA_FILE_NAME = 'libranzas_cnl_M202007.csv'
libranza_file_path = BASE_PATH_LIBRANZA+LIBRANZA_FILE_NAME

BASE_PATH_OUT_OBJECTS_LIBRANZA = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/'

In [3]:
month_number = ['0' + str(i) for i in range(1,10)] + ['10','11','12']
anio_mes_strs = ['2020-' + x for x in month_number if int(x) < 8]  ## Change for month number

In [4]:
import s3fs
import pyarrow.parquet as pq

fs = s3fs.S3FileSystem()

input_path = 's3://adl-refined-dev-popular/data_parquet/productos/libranzas/productos_libranzas_dwh_M202007'
input = input_path
dataset = pq.ParquetDataset(input, filesystem=fs)
lib = dataset.read()
lib = lib.to_pandas()

del dataset

In [5]:
# lib will be a global variable in this script
# lib = pd.read_csv(libranza_file_path,
#                   sep = ';',
#                   encoding = 'latin-1',
#                   na_values = '',
#                   dtype={'id_cliente': str}
#                  )

lib.rename(columns=lambda x: x.lower(), inplace=True)
lib_original_cols = lib.columns
print('Libranza original shape: ', lib.shape)

if lib['id_cliente'].isnull().sum() > 0:
    lib = lib.loc[lib['id_cliente'].notnull()]
    
lib['id_cliente'] =  lib['id_cliente'].astype(int)

Libranza original shape:  (3039505, 66)


In [6]:
lib.head()

,sk_cliente,id_cliente,sk_rc_libranza,id_rc_libranza,sk_fe_radicacion,sk_fe_aprobacion,sk_fe_desembolso,sk_fe_finalizacion,sk_fe_estado_actual,sk_convenio_libranza,sk_persona,dk_persona,sk_oficina,sk_producto_servicio,sk_clasificacion_tradicional,cd_oficina,cd_originador,ds_originador,cd_segmento_lib,ds_segmento_lib,cd_subsegmento_lib,ds_subsegmento_lib,no_solicitud,fe_solicitud,vl_monto_solicitado,mp_reestructurado,no_obligacion,fe_desembolso,vl_monto_aprobado,vl_recapitalizacion,vl_monto_desembolsado,no_cuotas,fe_finalizacion,no_meses_per_gracia,vl_total_cuota,fe_exigibilidad_cuota_1,cd_estado_actual,ds_estado_actual,fe_estado_actual,vl_tasa,vl_tasa_efectiva_anual,cd_modalidad_pag_int,ds_modalidad_pag_int,cd_periodicidad_pag_int,ds_periodicidad_pag_int,cd_base_liquidacion,cd_tipo_libranza,ds_tipo_libranza,cd_tipo_credito,ds_tipo_credito,cd_tipo_colocacion,ds_tipo_colocacion,no_obligacion_novada,vl_monto_novado,mp_compra_ind_cartera,mp_afecta_desp_nomina,cd_linea_credito,fe_carga,fe_actua,no_particion,cd_sector,cd_subsector,cd_tipo_pagaduria,ds_sector,ds_subsector,ds_tipo_pagaduria
273,60834,102652295584838601,1023924,1000003*11439693*LB*36403010026444,None,None,20040205,20060405,19000101,5063,662495,1000003*11439693,166,27,1,364,1,Banco Popular,1,Fuerzas Militares,4,Libranzas Ejercito Nacional,36403010028969,2004-02-05,2100000.0,0,36403010026444,2004-02-05,2100000.0,0.0,1555542.0,24,2006-04-05,0,113137.0,2004-04-05,2,Cancelada,1900-01-01,26.0,25.7097,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Novacion,1,1-ORD HST 50 AÃO 364 DIA,0,382878.0,0,0,66,NaT,NaT,0,1,4,1,ORDEN NACIONAL,Mindefensa Ejercito Nacional Activos,NACIONAL CENTRALIZADA
274,60834,102652295584838601,280423,1000003*11439693*LB*9550301121187,None,None,20010612,20040705,19000101,19724,662495,1000003*11439693,None,27,1,955,1,Banco Popular,1,Fuerzas Militares,4,Libranzas Ejercito Nacional,9550301121187,1900-01-01,3100000.0,0,9550301121187,2001-06-12,2960000.0,0.0,0.0,36,2004-07-05,0,128101.0,2001-07-05,2,Cancelada,1900-01-01,31.5,25.7097,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Nueva,1,1-ORD HST 50 AÃO 364 DIA,0,0.0,0,0,66,NaT,NaT,0,1,4,1,ORDEN NACIONAL,Mindefensa Ejercito Nacional Activos,NACIONAL CENTRALIZADA
275,60951,102652297314584801,215366,1000003*23823293*LB*2500300206080,None,None,20001227,20040205,19000101,5354,1250337,1000003*23823293,151,27,13,250,1,Banco Popular,4,Educativo,1,Libranzas Educadores,2500300206080,2000-12-27,2940000.0,0,2500300206080,2000-12-27,2940000.0,0.0,0.0,36,2004-02-05,0,125614.0,2001-02-05,2,Cancelada,1900-01-01,30.5,25.7097,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Nueva,1,1-ORD HST 50 AÃO 364 DIA,0,0.0,0,0,66,NaT,NaT,0,3,7,3,ENTES TERRITORIALES,Secretaría de Educacion,REGIONAL
276,60951,102652297314584801,390474,1000003*23823293*LB*25003010013115,None,None,20021003,20050705,19000101,5354,1250337,1000003*23823293,151,27,13,250,1,Banco Popular,4,Educativo,1,Libranzas Educadores,25003010013632,2002-09-27,4000000.0,0,25003010013115,2002-10-03,4000000.0,0.0,2104473.0,32,2005-07-05,0,175856.0,2002-11-05,2,Cancelada,1900-01-01,26.6,25.7097,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Novacion,1,1-ORD HST 50 AÃO 364 DIA,0,1697710.0,0,0,66,NaT,NaT,0,3,7,3,ENTES TERRITORIALES,Secretaría de Educacion,REGIONAL
277,60977,102652303039490201,197538,1000003*57441197*LB*4000300114523,None,None,20010608,20040705,20160630,16862,2299336,1000003*57441197,103,27,19,400,1,Banco Popular,0,Libranzas Otras,1,Libranzas Otras,4000300114523,2001-06-08,5000000.0,0,4000300114523,2001-06-08,5000000.0,0.0,0.0,36,2004-07-05,0,216387.0,2001-07-05,3,Castigado,2003-04-30,31.5,31.9009,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Nueva,1,1-ORD HST 50 AÃO 364 DIA,0,0.0,0,0,66,NaT,NaT,0,4,1,4,ENTES DESENTRALIZADOS,Salud,LOCAL


In [7]:
lib['ds_estado_actual'].value_counts()

Cancelada     2620846
En Cartera     356148
Castigado       55800
                 6438
Name: ds_estado_actual, dtype: int64

In [8]:
# This is the list of columns that we will drop in the first stage
lib_constant_cols = ['sk_producto_servicio', 'cd_modalidad_pag_int', 'ds_modalidad_pag_int',
                     'cd_periodicidad_pag_int', 'ds_periodicidad_pag_int',
                     'cd_base_liquidacion', 'no_obligacion_novada']

lib.drop(columns=lib_constant_cols, inplace=True)

In [9]:
lib = lib.loc[lib['ds_tipo_libranza'] == 'Organica']
lib = lib.loc[lib['ds_tipo_credito'].notnull()]

In [10]:
lib_date_cols = ['fe_solicitud', 'fe_desembolso', 'fe_finalizacion']
for col in lib_date_cols:
    lib[col] = pd.to_datetime(lib[col], format='%Y-%m-%d', errors='coerce')

In [11]:
## filtro tiempo
begining_training_period = '2006-01-01' # TODO: verify if this period is enough
ending_training_period = '2020-07-01'

In [12]:
period_filter = (lib['fe_solicitud'] >= begining_training_period) & (lib['fe_solicitud'] <= ending_training_period)
lib = lib.loc[period_filter]

In [13]:
lib_relevant_cols = ['id_cliente', 'sk_rc_libranza', 'fe_solicitud',
                     'vl_monto_solicitado', 'fe_desembolso', 'fe_finalizacion',
                     'vl_monto_aprobado', 'no_obligacion', 'vl_monto_desembolsado',
                     'no_cuotas', 'vl_total_cuota', 'ds_estado_actual',
                     'vl_tasa', 'ds_tipo_credito', 'ds_tipo_libranza', 'vl_monto_novado',
                    'ds_sector','ds_subsector','ds_tipo_pagaduria']

lib = lib[lib_relevant_cols]

In [14]:
lib.loc[lib['fe_solicitud'].isnull(), 'fe_solicitud'] = lib.loc[lib['fe_solicitud'].isnull(), 'fe_desembolso']

In [15]:
# def add_columns_to_lib():
hoy = pd.datetime.today()

lib['años_ult_lib'] = (hoy - lib['fe_desembolso'])/(np.timedelta64(1, 'D')*365)
lib['dias_procesamiento'] = (lib['fe_desembolso'] - lib['fe_solicitud'])/np.timedelta64(1, 'D')
# this is a necessary fix because there are many dates with fecha de solicitud in 1900
lib.loc[lib['dias_procesamiento'] > 360, 'dias_procesamiento'] = lib['dias_procesamiento'].median()
lib['prop_desemb_vs_aprobado'] = lib['vl_monto_desembolsado']/lib['vl_monto_aprobado']

In [16]:
lib.head()

,id_cliente,sk_rc_libranza,fe_solicitud,vl_monto_solicitado,fe_desembolso,fe_finalizacion,vl_monto_aprobado,no_obligacion,vl_monto_desembolsado,no_cuotas,vl_total_cuota,ds_estado_actual,vl_tasa,ds_tipo_credito,ds_tipo_libranza,vl_monto_novado,ds_sector,ds_subsector,ds_tipo_pagaduria,años_ult_lib,dias_procesamiento,prop_desemb_vs_aprobado
278,102652303931898101,1647562,2013-01-11,29500000.0,2013-01-14,2020-02-05,29500000.0,56503010074745,1499850.0,84,530269.0,Cancelada,12.60,Novacion,Organica,25926022.0,ORDEN NACIONAL,"OTROS POLICÍA, FONDO ROTATORIO ARMADA Y EJERCITO",NACIONAL CENTRALIZADA,7.596053,3.0,0.050842
279,102652303931898101,1433724,2013-05-15,32700000.0,2013-05-17,2020-06-05,32700000.0,50003010107799,1488409.0,84,587789.0,Cancelada,12.60,Novacion,Organica,28952184.0,ORDEN NACIONAL,"OTROS POLICÍA, FONDO ROTATORIO ARMADA Y EJERCITO",NACIONAL CENTRALIZADA,7.259066,2.0,0.045517
280,102652303931898101,2197892,2015-07-24,54500000.0,2015-07-30,2022-09-05,54500000.0,58703090012023,2180428.0,84,943608.0,En Cartera,10.68,Novacion,Organica,51640752.0,ORDEN NACIONAL,"OTROS POLICÍA, FONDO ROTATORIO ARMADA Y EJERCITO",NACIONAL CENTRALIZADA,5.056327,6.0,0.040008
281,102652303931898101,1052519,2006-09-11,7188264.0,2006-09-12,2011-11-05,7188000.0,47003010079423,1438836.0,60,192443.0,Cancelada,20.50,Novacion,Organica,5205064.0,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,13.941258,1.0,0.200172
284,102652303931898101,2006414,2013-12-24,38500000.0,2013-12-27,2021-02-05,38500000.0,58203010144269,1730882.0,84,718883.0,Cancelada,12.60,Novacion,Organica,36128628.0,ORDEN NACIONAL,"OTROS POLICÍA, FONDO ROTATORIO ARMADA Y EJERCITO",NACIONAL CENTRALIZADA,6.645368,3.0,0.044958


In [17]:
def prev_months_last_date(anio_mes_str):
    '''Gets the year and month of prediction and returns the previous month's last day's date'''
    anio, mes = anio_mes_str.split('-')
    anio = int(anio)
    mes = int(mes)
    if mes == 1:
        anio = int(anio - 1)
        mes = 12
    else:
        mes = mes - 1
    last_day_of_month = calendar.monthrange(anio, mes)[1]
    out = datetime(anio, mes, last_day_of_month)
    return out 

In [18]:
def get_grouped_df(anio_mes_str):
    '''Produces the extended lib_df and the grouping object that will be used
    as input to produce the final libranzas df'''
    cutoff_date = prev_months_last_date(anio_mes_str)
    ## considering records only before the predicting period, grouping by client
    lib_df = lib.loc[lib['fe_desembolso'] <= cutoff_date].sort_values(['id_cliente','fe_desembolso'], ascending=False).copy()
    gp = lib_df.groupby('id_cliente')
    return gp

In [19]:
def create_lib_df(gp):
    '''Gets the extended lib_df and the grouped object and returns the data frame ready for master table'''
    #producing the first dataframe
    lib_out = pd.DataFrame(index=gp.groups.keys()) # this is different from the actual prod code
    
    # number of records for each cliente
    lib_out['num_lib_solicitadas'] = gp.size()

    # these have a very close relation
    lib_out['prom_monto_solicitado'] = gp['vl_monto_solicitado'].mean()
    lib_out['prom_monto_aprobado'] = gp['vl_monto_aprobado'].mean()
    lib_out['prom_monto_desembolsado'] = gp['vl_monto_desembolsado'].mean()
    lib_out['prom_monto_novado'] = gp['vl_monto_novado'].mean()
    
    # Minimum years since las product
    lib_out['min_anos_ult_lib'] = gp['años_ult_lib'].min()

    # other averages
    lib_out['prom_prop_desembolso'] = gp['prop_desemb_vs_aprobado'].mean()
    lib_out['prom_n_cuotas'] = gp['no_cuotas'].mean()
    lib_out['prom_valor_cuota'] = gp['vl_total_cuota'].mean()
    lib_out['prom_tasa'] = gp['vl_tasa'].mean()

    # we have to choose one of these two...
    lib_out['prom_dias_procesamiento'] = gp['dias_procesamiento'].mean()
    lib_out['max_dias_procesamiento'] = gp['dias_procesamiento'].max()

    # counting novacion events within the historical period
    lib_out['n_novaciones'] = gp['ds_tipo_credito'].value_counts().unstack()['Novacion'].fillna(0).astype(int)
    # categorical columns
    
    lib_out['sector_ultlibranza'] = gp.apply(lambda x: x['ds_sector'].iloc[0])
    lib_out['subsector_ultlibranza'] = gp.apply(lambda x: x['ds_subsector'].iloc[0])
    lib_out['tipo_pagaduria_ultlibranza'] = gp.apply(lambda x: x['ds_tipo_pagaduria'].iloc[0])
    
    lib_out.index.rename('id_cliente', inplace=True)
    lib_out.reset_index(inplace = True)
    
    return lib_out

In [20]:
def get_lib_df(anio_mes_str):
    '''Glues the get_objects_for_lib_df and create_lib_df'''
    gp = get_grouped_df(anio_mes_str)
    out = create_lib_df(gp)
    return out

In [21]:
# producing the objects 
for ix, x in enumerate(anio_mes_strs):
    
    if ix == 0:
        lib_df = get_lib_df(x)
        lib_df['periodo'] = x
    else:
        lib_temp = get_lib_df(x)
        lib_temp['periodo'] = x
        lib_df = pd.concat([lib_df,lib_temp], ignore_index=True)
    
    print(x)

2020-01
2020-02
2020-03
2020-04
2020-05
2020-06
2020-07


In [22]:
lib.sort_values(['id_cliente','fe_solicitud'], ascending=False).head(20)

,id_cliente,sk_rc_libranza,fe_solicitud,vl_monto_solicitado,fe_desembolso,fe_finalizacion,vl_monto_aprobado,no_obligacion,vl_monto_desembolsado,no_cuotas,vl_total_cuota,ds_estado_actual,vl_tasa,ds_tipo_credito,ds_tipo_libranza,vl_monto_novado,ds_sector,ds_subsector,ds_tipo_pagaduria,años_ult_lib,dias_procesamiento,prop_desemb_vs_aprobado
3019202,999958035714320502,2719326,2018-08-08,97000000.0,2018-08-09,2026-11-05,97000000.0,25103260000664,96989280.0,96,1765068.0,Cancelada,13.080000,Nueva,Organica,0.0,PENSIONADOS,Fiduprevisora,NACIONAL CENTRALIZADA,2.026190,1.0,0.999889
2889793,999955693775109001,841646,2008-04-16,16000000.0,2008-04-17,2013-05-05,16000000.0,18003010294628,15215387.0,60,476588.0,Cancelada,25.740000,Nueva,Organica,0.0,PENSIONADOS,Caja De retiro de Las FFMM,NACIONAL CENTRALIZADA,12.343998,1.0,0.950962
2626796,999955266582332102,1423192,2011-02-03,34300000.0,2011-02-08,2018-01-08,34270000.0,01203010063300,10748344.0,84,780730.0,Cancelada,20.950001,Novacion,Organica,21462744.0,PENSIONADOS,I.S.S. Pensionados,NACIONAL CENTRALIZADA,9.530299,5.0,0.313637
2626799,999955266582332102,1645223,2009-07-14,27000000.0,2009-07-24,2014-07-05,27000000.0,01203010051965,21754520.0,60,785221.0,Cancelada,24.540001,Novacion,Organica,3872778.0,PENSIONADOS,I.S.S. Pensionados,NACIONAL CENTRALIZADA,11.075505,10.0,0.805723
2626797,999955266582332102,842517,2007-04-19,5570000.0,2007-04-30,2012-05-05,5570000.0,04203010024491,4517765.0,60,150061.0,Cancelada,20.799999,Novacion,Organica,766300.0,PENSIONADOS,I.S.S. Pensionados,NACIONAL CENTRALIZADA,13.311121,11.0,0.811089
2402512,999955266450872402,1761673,2014-06-27,29100000.0,2014-06-27,2021-07-05,29100000.0,35303070001146,28937310.0,84,533773.0,Cancelada,12.000000,Nueva,Organica,0.0,ENTES TERRITORIALES,Fondos Educativos,NACIONAL CENTRALIZADA,6.146738,0.0,0.994409
2402511,999955266450872402,1405539,2010-11-10,12000000.0,2010-11-12,2016-12-05,12000000.0,36303010059187,5050873.0,72,272880.0,Cancelada,17.879999,Novacion,Organica,6184277.0,ENTES TERRITORIALES,Fondos Educativos,NACIONAL CENTRALIZADA,9.771395,2.0,0.420906
2402513,999955266450872402,912371,2008-01-29,10000000.0,2008-01-30,2013-03-05,10000000.0,36303010037166,9389358.0,60,289773.0,Cancelada,24.360001,Nueva,Organica,0.0,ENTES TERRITORIALES,Fondos Educativos,NACIONAL CENTRALIZADA,12.557696,1.0,0.938936
2713296,999955266207457702,2059230,2014-02-17,31500000.0,2014-02-28,2021-11-05,31500000.0,08803400000086,4701647.0,92,595403.0,Castigado,12.000000,Novacion,Organica,26588540.0,PENSIONADOS,Otros Pensionados,NACIONAL CENTRALIZADA,6.472765,11.0,0.149259
2713297,999955266207457702,2200877,2014-01-20,26800000.0,2014-01-31,2021-10-05,26800000.0,08803350000886,21616128.0,92,499250.0,Cancelada,15.480000,Nueva,Organica,0.0,PENSIONADOS,Otros Pensionados,NACIONAL CENTRALIZADA,6.549477,11.0,0.806572


In [23]:
print(lib_df.shape)
lib_df.head()

(5482702, 18)


,id_cliente,num_lib_solicitadas,prom_monto_solicitado,prom_monto_aprobado,prom_monto_desembolsado,prom_monto_novado,min_anos_ult_lib,prom_prop_desembolso,prom_n_cuotas,prom_valor_cuota,prom_tasa,prom_dias_procesamiento,max_dias_procesamiento,n_novaciones,sector_ultlibranza,subsector_ultlibranza,tipo_pagaduria_ultlibranza,periodo
0,101052294884460201,2,7050000.0,6385000.0,5982935.0,0.0,11.508381,0.934040,72.0,164258.50,23.610001,3.50,6.0,0,PENSIONADOS,Fopep,NACIONAL CENTRALIZADA,2020-01
1,101052314032556001,1,70000000.0,70000000.0,69990040.0,0.0,1.242628,0.999858,72.0,1418040.00,10.800000,1.00,1.0,0,PENSIONADOS,Fiduprevisora,NACIONAL CENTRALIZADA,2020-01
2,101055260548785601,4,16175000.0,16175000.0,5261897.0,10197711.0,5.387833,0.402476,63.0,365051.75,13.590000,3.75,7.0,3,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-01
3,101055260650910902,4,12125000.0,12125000.0,7560881.0,4087683.5,4.807011,0.602487,60.0,279657.25,15.000000,3.25,7.0,3,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-01
4,101055260683554401,2,18000000.0,18000000.0,14080672.0,3618687.5,2.686464,0.831607,60.0,436291.00,15.480000,6.50,9.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-01


In [24]:
lib_df['periodo'].value_counts().sort_index()

2020-01    775372
2020-02    778205
2020-03    782149
2020-04    785455
2020-05    786076
2020-06    787022
2020-07    788423
Name: periodo, dtype: int64

In [25]:
lib_df['fecha_lib'] = pd.to_datetime(lib_df['periodo'],format='%Y-%m').dt.strftime('%Y%m').astype('int')

In [26]:
lib_df.to_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_libranzas_202001-202007",engine='pyarrow', index=False)

## Modelo

In [27]:
lib_df = pd.read_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_libranzas_201801_201912",engine='pyarrow')

In [28]:
lib_df[lib_df['id_cliente']==139955260696468901]

,id_cliente,num_lib_solicitadas,prom_monto_solicitado,prom_monto_aprobado,prom_monto_desembolsado,prom_monto_novado,min_anos_ult_lib,prom_prop_desembolso,prom_n_cuotas,prom_valor_cuota,prom_tasa,prom_dias_procesamiento,max_dias_procesamiento,n_novaciones,sector_ultlibranza,subsector_ultlibranza,tipo_pagaduria_ultlibranza,periodo,fecha_lib
30777,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-01,201801
729659,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-02,201802
1431496,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-03,201803
2137079,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-04,201804
2846053,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-05,201805
3558959,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-06,201806
4275834,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-07,201807
4995912,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-08,201808
5719182,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-09,201809
6446240,139955260696468901,2,15750000.0,15750000.0,9233777.0,5455780.0,7.124832,0.611499,72.0,326726.0,14.34,1.0,1.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-10,201810


In [29]:
lib[lib['id_cliente']==139955260696468901]

,id_cliente,sk_rc_libranza,fe_solicitud,vl_monto_solicitado,fe_desembolso,fe_finalizacion,vl_monto_aprobado,no_obligacion,vl_monto_desembolsado,no_cuotas,vl_total_cuota,ds_estado_actual,vl_tasa,ds_tipo_credito,ds_tipo_libranza,vl_monto_novado,ds_sector,ds_subsector,ds_tipo_pagaduria,años_ult_lib,dias_procesamiento,prop_desemb_vs_aprobado
87412,139955260696468901,1917182,2011-02-17,14500000.0,2011-02-18,2017-04-05,14500000.0,05703010033679,13475792.0,72,310395.0,Cancelada,15.48,Nueva,Organica,0.0,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,9.502902,1.0,0.929365
87413,139955260696468901,1818942,2013-05-09,17000000.0,2013-05-10,2019-06-05,17000000.0,01303480000097,4991762.0,72,343057.0,Cancelada,13.20,Novacion,Organica,10911560.0,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,7.278244,1.0,0.293633


In [30]:
path1 = 's3://adl-refined-dev-popular/data_orig/productos/libranzas/productos_libranzas_dwh_M202004.csv'
path2 = 's3://adl-refined-dev-popular/data_orig/productos/libranzas-t70-obligacion/productos_libranzas-t70-obligacion_cnl_M202004.txt'


lib1 = pd.read_csv(path1, sep=';', encoding = 'latin-1')
lib2 = pd.read_csv(path2, sep='|', encoding = 'latin-1')


print(lib1.shape)
print(lib2.shape)

(3168, 66)
(2697, 118)


In [31]:
lib1.head()

,SK_CLIENTE,ID_CLIENTE,SK_RC_LIBRANZA,ID_RC_LIBRANZA,SK_FE_RADICACION,SK_FE_APROBACION,SK_FE_DESEMBOLSO,SK_FE_FINALIZACION,SK_FE_ESTADO_ACTUAL,SK_CONVENIO_LIBRANZA,SK_PERSONA,DK_PERSONA,SK_OFICINA,SK_PRODUCTO_SERVICIO,SK_CLASIFICACION_TRADICIONAL,CD_OFICINA,CD_ORIGINADOR,DS_ORIGINADOR,CD_SEGMENTO_LIB,DS_SEGMENTO_LIB,CD_SUBSEGMENTO_LIB,DS_SUBSEGMENTO_LIB,NO_SOLICITUD,FE_SOLICITUD,VL_MONTO_SOLICITADO,MP_REESTRUCTURADO,NO_OBLIGACION,FE_DESEMBOLSO,VL_MONTO_APROBADO,VL_RECAPITALIZACION,VL_MONTO_DESEMBOLSADO,NO_CUOTAS,FE_FINALIZACION,NO_MESES_PER_GRACIA,VL_TOTAL_CUOTA,FE_EXIGIBILIDAD_CUOTA_1,CD_ESTADO_ACTUAL,DS_ESTADO_ACTUAL,FE_ESTADO_ACTUAL,VL_TASA,VL_TASA_EFECTIVA_ANUAL,CD_MODALIDAD_PAG_INT,DS_MODALIDAD_PAG_INT,CD_PERIODICIDAD_PAG_INT,DS_PERIODICIDAD_PAG_INT,CD_BASE_LIQUIDACION,CD_TIPO_LIBRANZA,DS_TIPO_LIBRANZA,CD_TIPO_CREDITO,DS_TIPO_CREDITO,CD_TIPO_COLOCACION,DS_TIPO_COLOCACION,NO_OBLIGACION_NOVADA,VL_MONTO_NOVADO,MP_COMPRA_IND_CARTERA,MP_AFECTA_DESP_NOMINA,CD_LINEA_CREDITO,FE_CARGA,FE_ACTUA,NO_PARTICION,CD_SECTOR,CD_SUBSECTOR,CD_TIPO_PAGADURIA,DS_SECTOR,DS_SUBSECTOR,DS_TIPO_PAGADURIA
0,-1,-1,10658363,1000003*4437701*LB*01403070007748,-1,-1,20200331,20300705,20200430,17239,-1,1000003*4437701,67,-1,1,14,1,Banco Popular,1,Fuerzas Militares,4,Libranzas Ejercito Nacional,1403070011653,2020-03-18,128000000.0,0,1403070007748,2020-03-31,128000000.0,3870341.0,127988106.0,120,2030-07-05,2,2056637.0,2020-07-05,1,En Cartera,2020-04-30,13.08,13.8933,V,Vencido,M,Mensual,360,1,Organica,1.0,Nueva,7,7-ATR HST 50 AÃO 364 DIA,0,0.0,1,1,66,2020-04-01 04:49:46,2020-05-05 20:44:11,1,1,4,1,ORDEN NACIONAL,Mindefensa Ejercito Nacional Activos,NACIONAL CENTRALIZADA
1,-1,-1,10671192,1000003*74376005*LB*26103070001806,-1,-1,20200423,20300705,20200430,26661,-1,1000003*74376005,153,-1,11,261,1,Banco Popular,3,Pensionados,5,Libranzas Caja Retiro FFMM,26103070002716,2020-04-17,47000000.0,0,26103070001806,2020-04-23,47000000.0,0.0,46686530.0,120,2030-07-05,0,659097.0,2020-05-05,1,En Cartera,2020-04-30,10.68,11.2186,V,Vencido,M,Mensual,360,1,Organica,1.0,Nueva,7,7-ATR HST 50 AÃO 364 DIA,0,0.0,1,0,66,2020-04-24 05:14:48,2020-05-05 20:44:11,1,2,5,1,PENSIONADOS,Caja De retiro de Las FFMM,NACIONAL CENTRALIZADA
2,-1,-1,10660051,1000003*1103111540*LB*20503470001754,-1,-1,20200402,20220605,20200430,12187,-1,1000003*1103111540,216,-1,6,205,1,Banco Popular,2,Policia Nacional,1,Libranzas Policia Nacional,20503470002325,2020-03-31,10000000.0,0,20503470001754,2020-04-02,10000000.0,0.0,9730585.0,24,2022-06-05,0,493654.0,2020-05-05,1,En Cartera,2020-04-30,16.08,17.3197,V,Vencido,M,Mensual,360,1,Organica,1.0,Nueva,47,47-ORD PATRU 50 AÃO 364 DIA,0,0.0,0,0,66,2020-04-03 05:23:08,2020-05-05 20:44:11,1,1,6,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA
3,-1,-1,10658250,1000003*32639300*LB*04503240006082,-1,-1,20200331,20300605,20200430,19318,-1,1000003*32639300,5,-1,7,45,1,Banco Popular,3,Pensionados,1,Libranzas Colpensiones,4503240007189,2020-03-16,30200000.0,0,4503240006082,2020-03-31,30200000.0,1103288.0,30197189.0,120,2030-06-05,2,505063.0,2020-06-05,1,En Cartera,2020-04-30,13.80,14.7072,V,Vencido,M,Mensual,360,1,Organica,1.0,Nueva,24,24-DE 51 < 69 AÃO 364 DIA,0,0.0,1,1,66,2020-04-01 04:49:46,2020-05-05 20:44:11,1,2,1,1,PENSIONADOS,I.S.S. Pensionados,NACIONAL CENTRALIZADA
4,-1,-1,10671744,1000003*46376921*LB*27003070007827,-1,-1,20200424,20300605,20200430,15939,-1,1000003*46376921,92,-1,13,270,1,Banco Popular,4,Educativo,1,Libranzas Educadores,27003070010742,2020-03-19,104000000.0,0,27003070007827,2020-04-24,104000000.0,1510740.0,103998282.0,120,2030-06-05,1,1427024.0,2020-06-05,1,En Cartera,2020-04-30,9.60,10.0339,V,Vencido,M,Mensual,360,1,Organica,1.0,Nueva,7,7-ATR HST 50 AÃO 364 DIA,0,0.0,1,1,66,2020-04-25 04:45:45,2020-05-05 20:44:11,1,3,7,3,ENTES TERRITORIALES,Secretaría de Educacion,REGIONAL


In [32]:
lib2.head()

,C70IDOBLIGACION,C70LINEA,C70MODALIDAD,C70REGIMEN,C70TIPODOCTERCERO,C70IDTERCERO,C70PAGADURIA,C70CODSUCURSAL,C70OFIRECEP,C70OFIRECEPREAL,C70OFIVENTA,C70FECVENTA,C70FECRECOMPRA,C70FRECCUOTA,C70CUOTASPACTADAS,C70CUOTASPAGADAS,C70CUOTASCOBRADAS,C70MONTOAPROB,C70NROCTADESEM,C70FORMADESEM,C70TASANOMAPROB,C70TASAEFECAPROB,C70IDSEGURO,C70VALCUOTA,C70COMISION,C70IVA,C70DIASAJUSTE,C70INTANT,C70INTANTCAUSADO,C70INTANTXCAUSAR,C70INTACAUSAR,C70INTCAUSADO,C70SEGURO,C70NETODESEMBOLSADO,C70COMPROBANTEDESEM,C70FECCAUSACION,C70FECHADESEM,C70FECINI,C70FECFIN,C70FECANTCUOTA,C70FECPROXCUOTA,C70FECINIMORA,C70FECINIMORAANT,C70FECULTPAGO,C70ULTCOMPROBANTE,C70REVERSARCAUSACION,C70FECININOVEDAD,C70FECFINNOVEDAD,C70SALDOCAPITAL,C70SALDOCAPITALV,C70INTPAGADO,C70SALDOINTCTE,C70SALDOINTCTEV,C70SALDOINTMORA,C70SALDOINTCTGTES,C70SALDOINTMORACTGTES,C70SALDOAFAVOR,C70PROVISIONCAPITAL,C70PROVISIONCAPITALANT,C70PROVISIONINTCTE,C70PROVISIONINTCTEANT,C70PROVISIONINTMORA,C70PROVISIONINTMORAANT,C70FALTANTECREDITO,C70APROVECHAMIENTOS,C70SOBRANTES,C70TIPOTASACTEACT,C70TIPOTASACTEANT,C70TIPOTASAMORAACT,C70TIPOTASAMORAANT,C70VALORTASACTEACT,C70VALORTASACTEANT,C70VALORTASAMORAACT,C70VALORTASAMORAANT,C70NOVACION,C70VALORNOVADO,C70ORIGEN,C70CALIFICACION,C70IDSOLICITUD,C70TRXDESEM,C70FECCIERRE,C70ESTADO,C70CALIFICACIONXRIESGO,C70CALIFICACIONANT,C70MARCASOBRANTE,C70FECINICBRJUR,C70COBROJURIDICO,C70FECCAMBIOESTADO,C70GASTOSPAGADOS,C70GASTOSCOBRADOS,C70GRUPO,C70PLAZOMAXTASA,C70MONTOMAXTASA,C70EXONERADO,C70VALOREXONERACION,C70RESTRUCTURADA,C70TIPOSEGURO,C70VALORTOTALSEGURO,C70NROCUOTASADIFERIR,C70VALORSEGUROFINANCIAR,C70VALORSEGUROANTICIPADO,C70CUOTASPAGADASSEG,C70SALDOSEGURO,C70ESTADOCXC,C70FACTORSEGURO,C70SALDOCXC,C70TIPO_CUENTA,C70CTADESEM_NRO,C70COD_BANCO,C70COMISION_ACH,C70IVA_ACH,C70DISMINUYE_CUOTA,C70DESPRENDIBLENOMINA,C70MESESDEGRACIA,C70_REINCIDENCIA,C70MARCA_REE_MOD,ID_CLIENTE,c70tipodoctercero_HOM
0,18503070010979,3,7,1,1,97471534,8001413975,432,185,185,NaN,NaN,NaN,30,120,0,1,56500000,230182766865,2,10.68,11.2186,1,812128,99000,18810,71,1172476,613238,559238,515276,0,108555,56497383,185-03073070603,31/05/2020,24/04/2020,05/07/2020,05/07/2030,05/07/2020,05/08/2020,05/08/2020,NaN,05/07/2020,0,0,01/06/2020,01/05/2030,57896224.0,0,0,515276,0,0,0,0,0,1626884,0,0,0,0,0,0,0,0,FIJA,FIJA,MORA,MORA,11.2186,11.2186,27.13,27.235,0,0,FSII,A,18503070014993,10041,31/05/2020,70,A,NaN,0,01/05/2001,0,30/04/2020,0,0,8U1,120,309000000,S,57896224,0,4,2722487,NaN,2613932,108555,0,2613932,0,0.0075,38831,0,-,0,0,0,0,1,1,0,NaN,NaN,1000003
1,26503680000391,3,68,2,1,10175755,8200014059,1,265,265,NaN,NaN,NaN,30,108,0,1,58500000,230265100560,2,13.08,13.8933,1,949988,99000,18810,11,232078,232078,0,106755,530893,36563,6999506,265-03680001930,31/05/2020,24/04/2020,05/05/2020,05/06/2029,05/06/2020,05/06/2020,05/06/2020,NaN,05/05/2020,0,0,01/05/2020,01/04/2029,58500000.0,0,0,637648,0,0,0,0,0,1643850,0,13538,0,0,0,0,0,0,FIJA,FIJA,MORA,MORA,13.8933,13.8933,27.13,27.235,1,51114043,FSII,A,26503680000499,10041,31/05/2020,70,NaN,NaN,0,01/05/2001,0,30/04/2020,0,0,88D,108,221315100,S,58500000,0,4,2452671,NaN,2416108,36563,0,2416108,0,0.0075,176027,0,-,0,0,0,0,0,0,0,NaN,NaN,1000003
2,30003680000931,3,68,2,1,8778069,8001413975,261,300,300,NaN,NaN,NaN,30,96,0,1,95000000,230300802766,2,13.08,13.8933,1,1641556,99000,18810,41,1397146,1230087,167059,1035497,0,118750,16461050,300-03680005524,31/05/2020,24/04/2020,05/06/2020,05/07/2028,05/07/2020,05/07/2020,05/07/2020,NaN,05/06/2020,0,0,01/06/2020,01/05/2028,95000000.0,0,0,1035497,0,0,0,0,0,2669500,0,0,0,0,0,0,0,0,FIJA,FIJA,MORA,MORA,13.8933,13.8933,27.13,27.235,1,76905244,FSII,A,30003680001075,10041,31/05/2020,70,NaN,NaN,0,01/05/2001,0,30/04/2020,0,0,43C,120,263340900,S,95000000,0,4,3547932,NaN,3429182,118750,0,3429182,0,0.0075,82771,0,-,0,0,0,0,0,0,0,NaN,NaN,1000003
3,30003070014715,3,7,2,1,36623878,8923999991,4,300,300,NaN,NaN,NaN,30,120,0,1,96000000,230300228582,2,10.68,11.2186,1,1381077,99000,18810,41,1155503,1031593,123910,866800,0,121742,95998258,300-03073340

In [33]:
lib2.count()

C70IDOBLIGACION             2697
C70LINEA                    2697
C70MODALIDAD                2697
C70REGIMEN                  2697
C70TIPODOCTERCERO           2697
C70IDTERCERO                2697
C70PAGADURIA                2697
C70CODSUCURSAL              2697
C70OFIRECEP                 2697
C70OFIRECEPREAL             2697
C70OFIVENTA                    0
C70FECVENTA                    0
C70FECRECOMPRA                 0
C70FRECCUOTA                2697
C70CUOTASPACTADAS           2697
C70CUOTASPAGADAS            2697
C70CUOTASCOBRADAS           2697
C70MONTOAPROB               2697
C70NROCTADESEM              2697
C70FORMADESEM               2697
C70TASANOMAPROB             2697
C70TASAEFECAPROB            2697
C70IDSEGURO                 2697
C70VALCUOTA                 2697
C70COMISION                 2697
C70IVA                      2697
C70DIASAJUSTE               2697
C70INTANT                   2697
C70INTANTCAUSADO            2697
C70INTANTXCAUSAR            2697
C70INTACAU

In [34]:
lib1.describe()

,SK_CLIENTE,ID_CLIENTE,SK_RC_LIBRANZA,SK_FE_RADICACION,SK_FE_APROBACION,SK_FE_DESEMBOLSO,SK_FE_FINALIZACION,SK_FE_ESTADO_ACTUAL,SK_CONVENIO_LIBRANZA,SK_PERSONA,SK_OFICINA,SK_PRODUCTO_SERVICIO,SK_CLASIFICACION_TRADICIONAL,CD_OFICINA,CD_ORIGINADOR,CD_SEGMENTO_LIB,CD_SUBSEGMENTO_LIB,NO_SOLICITUD,VL_MONTO_SOLICITADO,MP_REESTRUCTURADO,NO_OBLIGACION,VL_MONTO_APROBADO,VL_RECAPITALIZACION,VL_MONTO_DESEMBOLSADO,NO_CUOTAS,NO_MESES_PER_GRACIA,VL_TOTAL_CUOTA,CD_ESTADO_ACTUAL,VL_TASA,VL_TASA_EFECTIVA_ANUAL,CD_BASE_LIQUIDACION,CD_TIPO_LIBRANZA,CD_TIPO_CREDITO,CD_TIPO_COLOCACION,NO_OBLIGACION_NOVADA,VL_MONTO_NOVADO,MP_COMPRA_IND_CARTERA,MP_AFECTA_DESP_NOMINA,CD_LINEA_CREDITO,NO_PARTICION,CD_SECTOR,CD_SUBSECTOR,CD_TIPO_PAGADURIA
count,3.168000e+03,3.168000e+03,3.168000e+03,3168.0,3168.0,3.168000e+03,3.168000e+03,3.168000e+03,3168.000000,3.168000e+03,3168.000000,3168.0,3168.000000,3168.000000,3168.0,3168.000000,3168.000000,3.168000e+03,3.168000e+03,3168.000000,3.168000e+03,3.168000e+03,3.168000e+03,3.168000e+03,3168.000000,3168.000000,3.168000e+03,3168.000000,3168.000000,3168.000000,3168.0,3168.0,3168.000000,3168.000000,3168.0,3.168000e+03,3168.000000,3168.000000,3168.0,3168.000000,3168.000000,3168.000000,3168.000000
mean,2.142760e+06,5.531412e+17,1.066627e+07,-1.0,-1.0,2.020040e+07,2.028366e+07,2.020038e+07,15825.842803,1.838014e+06,134.857955,-1.0,8.620896,332.527778,1.0,2.858586,2.184028,3.325623e+13,4.742428e+07,0.002210,3.325623e+13,4.742428e+07,3.881259e+05,2.730230e+07,99.695707,0.327967,8.275041e+05,1.001894,13.753485,14.682308,360.0,1.0,1.563763,45.448548,0.0,1.977114e+07,0.259785,0.200758,66.0,0.998106,1.960859,5.424558,1.308396
std,1.086706e+06,2.610116e+17,8.016784e+03,0.0,0.0,3.575502e+01,2.534426e+04,3.374787e+03,8725.054655,9.936139e+05,78.204989,0.0,3.210030,196.662830,0.0,0.750675,1.760503,1.966629e+13,4.567041e+07,0.046962,1.966629e+13,4.567041e+07,1.059094e+06,3.970371e+07,30.391554,0.689183,6.886405e+05,0.043485,2.326748,2.648336,0.0,0.0,0.495996,29.354037,0.0,3.086494e+07,0.438586,0.400630,0.0,0.043485,0.551470,6.266877,0.724791
min,-1.000000e+00,-1.000000e+00,1.065770e+07,-1.0,-1.0,2.020033e+07,2.021050e+07,2.001050e+07,9.000000,-1.000000e+00,-1.000000,-1.0,1.000000,9.000000,1.0,0.000000,1.000000,9.032400e+11,1.150000e+06,0.000000,9.032400e+11,1.150000e+06,0.000000e+00,5.721000e+04,12.000000,0.000000,2.153300e+04,1.000000,7.200000,7.442400,360.0,1.0,1.000000,1.000000,0.0,0.000000e+00,0.000000,0.000000,66.0,0.000000,1.000000,1.000000,1.000000
25%,1.237185e+06,3.360523e+17,1.065993e+07,-1.0,-1.0,2.020040e+07,2.027060e+07,2.020043e+07,8556.000000,9.997510e+05,63.000000,-1.0,7.000000,182.000000,1.0,3.000000,1.000000,1.820355e+13,1.900000e+07,0.000000,1.820355e+13,1.900000e+07,0.000000e+00,6.044478e+06,84.000000,0.000000,3.817990e+05,1.000000,11.880000,12.548700,360.0,1.0,1.000000,23.000000,0.0,0.000000e+00,0.000000,0.000000,66.0,1.000000,2.000000,1.000000,1.000000
50%,2.140742e+06,5.443523e+17,1.066446e+07,-1.0,-1.0,2.020041e+07,2.030060e+07,2.020043e+07,15136.000000,1.742442e+06,135.000000,-1.0,7.000000,350.500000,1.0,3.000000,1.000000,3.505344e+13,3.540000e+07,0.000000,3.505344e+13,3.540000e+07,0.000000e+00,1.316322e+07,120.000000,0.000000,6.660885e+05,1.000000,13.800000,14.707200,360.0,1.0,2.000000,44.000000,0.0,6.428844e+06,0.000000,0.000000,66.0,1.000000,2.000000,4.000000,1.000000
75%,3.066888e+06,7.840523e+17,1.067097e+07,-1.0,-1.0,2.020042e+07,2.030070e+07,2.020043e+07,23285.000000,2.632986e+06,211.000000,-1.0,11.000000,484.000000,1.0,3.000000,3.000000,4.840324e+13,6.213475e+07,0.000000,4.840324e+13,6.213475e+07,0.000000e+00,3.181089e+07,120.000000,0.000000,1.074600e+06,1.000000,15.480000,16.626900,360.0,1.0,2.000000,70.000000,0.0,2.870889e+07,1.000000,0.000000,66.0,1.000000,2.000000,6.000000,1.000000
max,3.838157e+06,9.990582e+17,1.069042e+07,-1.0,-1.0,2.020043e+07,2.030080e+07,2.020052e+07,30368.000000,3.519419e+06,257.000000,-1.0,19.000000,720.000000,1.0,5.000000,6.000000,7.200386e+13,5.900000e+08,1.000000,7.200386e

In [35]:
lib2.describe()

,C70IDOBLIGACION,C70LINEA,C70MODALIDAD,C70REGIMEN,C70TIPODOCTERCERO,C70IDTERCERO,C70PAGADURIA,C70CODSUCURSAL,C70OFIRECEP,C70OFIRECEPREAL,C70OFIVENTA,C70FECVENTA,C70FECRECOMPRA,C70FRECCUOTA,C70CUOTASPACTADAS,C70CUOTASPAGADAS,C70CUOTASCOBRADAS,C70MONTOAPROB,C70FORMADESEM,C70TASANOMAPROB,C70TASAEFECAPROB,C70IDSEGURO,C70VALCUOTA,C70COMISION,C70IVA,C70DIASAJUSTE,C70INTANT,C70INTANTCAUSADO,C70INTANTXCAUSAR,C70INTACAUSAR,C70INTCAUSADO,C70SEGURO,C70NETODESEMBOLSADO,C70FECINIMORAANT,C70REVERSARCAUSACION,C70SALDOCAPITAL,C70SALDOCAPITALV,C70INTPAGADO,C70SALDOINTCTE,C70SALDOINTCTEV,C70SALDOINTMORA,C70SALDOINTCTGTES,C70SALDOINTMORACTGTES,C70SALDOAFAVOR,C70PROVISIONCAPITAL,C70PROVISIONCAPITALANT,C70PROVISIONINTCTE,C70PROVISIONINTCTEANT,C70PROVISIONINTMORA,C70PROVISIONINTMORAANT,C70FALTANTECREDITO,C70APROVECHAMIENTOS,C70SOBRANTES,C70VALORTASACTEACT,C70VALORTASACTEANT,C70VALORTASAMORAACT,C70VALORTASAMORAANT,C70NOVACION,C70VALORNOVADO,C70IDSOLICITUD,C70TRXDESEM,C70ESTADO,C70CALIFICACIONANT,C70MARCASOBRANTE,C70COBROJURIDICO,C70GASTOSPAGADOS,C70GASTOSCOBRADOS,C70PLAZOMAXTASA,C70MONTOMAXTASA,C70VALOREXONERACION,C70RESTRUCTURADA,C70TIPOSEGURO,C70VALORTOTALSEGURO,C70NROCUOTASADIFERIR,C70VALORSEGUROFINANCIAR,C70VALORSEGUROANTICIPADO,C70CUOTASPAGADASSEG,C70SALDOSEGURO,C70ESTADOCXC,C70FACTORSEGURO,C70SALDOCXC,C70TIPO_CUENTA,C70COD_BANCO,C70COMISION_ACH,C70IVA_ACH,C70DISMINUYE_CUOTA,C70DESPRENDIBLENOMINA,C70MESESDEGRACIA,C70_REINCIDENCIA,C70MARCA_REE_MOD,ID_CLIENTE,c70tipodoctercero_HOM
count,2.697000e+03,2697.0,2697.000000,2697.000000,2697.0,2.697000e+03,2.697000e+03,2697.000000,2697.000000,2697.000000,0.0,0.0,0.0,2697.0,2697.000000,2697.000000,2697.000000,2.697000e+03,2697.000000,2697.000000,2697.000000,2697.0,2.697000e+03,2697.0,2697.000000,2697.000000,2.697000e+03,2.697000e+03,2.697000e+03,2.697000e+03,2.697000e+03,2.697000e+03,2.697000e+03,0.0,2697.0,2.697000e+03,2697.0,2.697000e+03,2.697000e+03,2697.0,2697.0,2697.0,2697.0,2.697000e+03,2.697000e+03,2697.0,2697.000000,2697.0,2697.0,2697.0,2697.000000,2697.000000,2697.000000,2697.000000,2697.000000,2697.000000,2697.000000,2697.000000,2.697000e+03,2.697000e+03,2697.000000,2697.000000,0.0,2697.0,2697.000000,2697.0,2697.0,2697.000000,2.697000e+03,2.697000e+03,2697.000000,2697.000000,2.697000e+03,0.0,2.697000e+03,2.697000e+03,2697.0,2.697000e+03,2697.0,2697.000000,2.697000e+03,2697.000000,2697.000000,2697.0,2697.0,2697.0,2697.000000,2697.000000,2697.000000,0.0,0.0,2697.0
mean,3.397869e+13,3.0,45.969225,1.304783,1.0,1.433676e+08,8.731577e+09,221.816463,339.752317,339.752317,NaN,NaN,NaN,30.0,99.842047,0.091583,1.091212,4.787413e+07,2.011494,13.805161,14.740513,1.0,8.333705e+05,99000.0,18621.690768,26.760104,4.888341e+05,3.986440e+05,9.029244e+04,2.352834e+05,2.667616e+05,1.049435e+05,2.640371e+07,NaN,0.0,4.788330e+07,0.0,4.003091e+04,5.020450e+05,0.0,0.0,0.0,0.0,4.287543e+05,1.515304e+06,0.0,6789.144605,0.0,0.0,0.0,12.238042,1.754913,382.311828,14.740513,14.740513,27.130667,27.237781,0.589544,2.109415e+07,3.397869e+13,10018.507972,70.107527,NaN,0.0,0.022618,0.0,0.0,111.090842,2.753453e+08,4.821229e+07,0.004820,3.998888,5.451696e+06,NaN,5.343139e+06,1.049435e+05,0.0,5.333645e+06,0.0,0.022099,1.209846e+05,0.022989,0.286615,0.0,0.0,0.0,0.184279,0.296255,0.002595,NaN,NaN,1000003.0
std,1.929741e+13,0.0,30.155879,0.460401,0.0,3.251214e+08,4.038946e+08,228.862121,192.974109,192.974109,NaN,NaN,NaN,0.0,30.418524,0.289773,0.287964,4.676409e+07,0.106613,2.310461,2.629820,0.0,7.041997e+05,0.0,1872.948738,22.348980,7.625804e+05,5.221546e+05,3.139028e+05,3.687637e+05,3.112901e+05,4.060902e+05,4.021578e+07,NaN,0.0,4.713768e+07,0.0,1.600333e+05,4.180883e+05,0.0,0.0,0.0,0.0,5.943255e+05,4.810977e+06,0.0,7932.814988,0.0,0.0,0.0,455.958540,91.137273,17849.600543,2.629820,2.629820,0.017536,0.045593,0.492008,3.161667e+07,1.929741e+13,26.192353,1.762915,NaN,0.0,0.148709,0.0,0.0,21.438328,6.715672e+08,4.733575e+07,0.069273,0.057767,1.049778e+07,NaN,1.029738e+07,4.060902e+05,0.0,1.027799e+07,0.0,0.027488,2.217791e+05,0.213226,3.

In [36]:
''

''

In [37]:
lib_df = pd.read_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_libranzas_201801_201912",engine='pyarrow')
lib_df.count()

id_cliente                    17696740
num_lib_solicitadas           17696740
prom_monto_solicitado         17696740
prom_monto_aprobado           17696740
prom_monto_desembolsado       17696740
prom_monto_novado             17696740
min_anos_ult_lib              17696740
prom_prop_desembolso          17696740
prom_n_cuotas                 17696740
prom_valor_cuota              17696740
prom_tasa                     17696740
prom_dias_procesamiento       17696740
max_dias_procesamiento        17696740
n_novaciones                  17696740
sector_ultlibranza            17696740
subsector_ultlibranza         17696740
tipo_pagaduria_ultlibranza    17696740
periodo                       17696740
fecha_lib                     17696740
dtype: int64

In [38]:
lib_df['sector_ultlibranza'].value_counts()

PENSIONADOS              9913414
ORDEN NACIONAL           4041447
ENTES TERRITORIALES      3141400
PRIVADO                   320500
ENTES DESENTRALIZADOS     279979
Name: sector_ultlibranza, dtype: int64

In [39]:
# Creando Data Libranza
lib_df['fecha_lib'] = pd.to_datetime(lib_df['periodo'], format='%Y-%m').dt.strftime('%Y%m').astype('int')
lib_df.head(20)

,id_cliente,num_lib_solicitadas,prom_monto_solicitado,prom_monto_aprobado,prom_monto_desembolsado,prom_monto_novado,min_anos_ult_lib,prom_prop_desembolso,prom_n_cuotas,prom_valor_cuota,prom_tasa,prom_dias_procesamiento,max_dias_procesamiento,n_novaciones,sector_ultlibranza,subsector_ultlibranza,tipo_pagaduria_ultlibranza,periodo,fecha_lib
0,101052294884460201,2,7.050000e+06,6.385000e+06,5.982935e+06,0.000000e+00,11.354969,0.934040,72.000000,164258.500000,23.610000,3.500000,6.0,0,PENSIONADOS,Fopep,NACIONAL CENTRALIZADA,2018-01,201801
1,101055260548785601,4,1.617500e+07,1.617500e+07,5.261897e+06,1.019771e+07,5.234421,0.402476,63.000000,365051.750000,13.590000,3.750000,7.0,3,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-01,201801
2,101055260650910902,4,1.212500e+07,1.212500e+07,7.560881e+06,4.087684e+06,4.653599,0.602487,60.000000,279657.250000,15.000000,3.250000,7.0,3,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-01,201801
3,101055260683554401,2,1.800000e+07,1.800000e+07,1.408067e+07,3.618688e+06,2.533051,0.831607,60.000000,436291.000000,15.480000,6.500000,9.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2018-01,201801
4,101055260720859701,2,8.250000e+06,8.250000e+06,5.983176e+06,1.797331e+06,10.423462,0.745965,66.000000,202629.500000,19.214800,3.000000,5.0,1,ENTES TERRITORIALES,Secretaría de Educacion,REGIONAL,2018-01,201801
5,101055266255810701,1,1.000000e+07,1.000000e+07,9.530928e+06,0.000000e+00,13.157708,0.953093,60.000000,269410.000000,20.800000,13.000000,13.0,0,PENSIONADOS,I.S.S. Pensionados,NACIONAL CENTRALIZADA,2018-01,201801
6,101055266367157902,3,2.713333e+07,2.713333e+07,1.533572e+07,1.125101e+07,2.965928,0.618465,98.666667,563069.666667,17.120000,9.000000,14.0,2,PENSIONADOS,I.S.S. Pensionados,NACIONAL CENTRALIZADA,2018-01,201801
7,101055266603150201,2,3.250000e+06,3.250000e+06,2.337335e+06,8.363075e+05,3.064558,0.785358,36.000000,118828.000000,18.480000,10.500000,14.0,1,PENSIONADOS,I.S.S. Pensionados,NACIONAL CENTRALIZADA,2018-01,201801
8,101055266686923002,5,1.786000e+07,1.786000e+07,1.006541e+07,6.742780e+06,3.426202,0.671437,62.400000,458791.800000,20.576000,2.000000,5.0,3,PENSIONADOS,Fiduprevisora,NACIONAL CENTRALIZADA,2018-01,201801
9,101055629594214801,2,8.538500e+06,8.538500e+06,6.333382e+06,2.091991e+06,2.579626,0.783307,54.000000,232828.500000,17.580000,7.500000,12.0,1,PENSIONADOS,I.S.S. Pensionados,NACIONAL CENTRALIZADA,2018-01,201801


In [40]:
data = pd.read_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_activos_201801_202003_v2", engine='pyarrow')
data.head()

,tipo_ident,sk_cliente,id_cliente,tipo_producto,lineacredi,nrooblig,cod_ofic,plazo,cuota_paga,fecdes,cupo_aprob,fecven,vlr_desemb,sal_capita,vlr_cuota,tasint_cte,tasint_mor,dias_morak,estado,calif_cart,fecha,fecha_activo,ano_po,SMLV,tipo_prod,fecha_desembolso,fecha_obs,apertura_1meses,apertura_3meses,apertura_6meses,apertura_12meses,edad_credito_meses,cuenta_vigente,altura_mora,utilizacion,amortizacion
0,1.0,541607.0,101052294884460201,TC,62,00660004000000240385,694,730,0.0,13/06/2015,1557000.0,01/05/2022,1120445.23,1085925.92,219304.71,28.69,28.73,0,ACTIVO,A,201901,201901,2019,828116.0,TC_X,2015-06-13,2019-01-01,0,0,0,0,55,1,aldia,0.969191,0.030809
1,1.0,541607.0,101052294884460201,TC,62,00660004000000240385,694,730,0.0,13/06/2015,1557000.0,01/05/2022,903920.19,845803.58,270595.74,29.01,29.05,0,ACTIVO,A,201902,201902,2019,828116.0,TC_X,2015-06-13,2019-02-01,0,0,0,0,56,1,aldia,0.935706,0.064294
2,1.0,541607.0,101052294884460201,TC,62,00660004000000240385,694,730,0.0,13/06/2015,1557000.0,01/05/2022,829474.61,786827.70,160811.21,29.01,29.05,0,ACTIVO,A,201903,201903,2019,828116.0,TC_X,2015-06-13,2019-03-01,0,0,0,0,57,1,aldia,0.948586,0.051414
3,1.0,541607.0,101052294884460201,TC,62,00660004000000240385,694,730,0.0,13/06/2015,1557000.0,01/05/2022,1313152.94,1272469.46,193884.92,28.93,28.96,0,ACTIVO,A,201904,201904,2019,828116.0,TC_X,2015-06-13,2019-04-01,0,0,0,0,58,1,aldia,0.969018,0.030982
4,1.0,541607.0,101052294884460201,TC,62,00660004000000240385,694,730,0.0,13/06/2015,1557000.0,01/05/2022,1253627.06,1208982.40,221613.61,28.90,28.94,0,ACTIVO,A,201905,201905,2019,828116.0,TC_X,2015-06-13,2019-05-01,0,0,0,0,59,1,aldia,0.964388,0.035612


In [41]:
print(data.shape)
print(data.drop_duplicates().shape)

(9610068, 36)
(9610068, 36)


In [42]:
prueba[prueba['fecha_activo']==202005]

NameError: name 'prueba' is not defined

In [ ]:
(data['id_cliente'].isnull()).sum()

In [ ]:
prueba = data[data['id_cliente']==761552294082694601]

prueba['fecha_activo'].value_counts()


In [ ]:
data['id_cliente'].value_counts().head(15)

In [ ]:
data